__Probing Language Models__

This notebook serves as a start for your NLP2 assignment on probing Language Models. This notebook will become part of the contents that you will submit at the end, so make sure to keep your code (somewhat) clean :-)

__note__: This assignment is not dependent on big fancy GPUs. I run all this stuff on my own 3 year old CPU, without any Colab hassle. So it's up to you to decide how you want to run it.

# Models

For the Transformer models you are advised to make use of the `transformers` library of Huggingface: https://github.com/huggingface/transformers
Their library is well documented, and they provide great tools to easily load in pre-trained models.

In [1]:
#
## Your code for initializing the transformer model(s)
#
# Note that transformer models use their own `tokenizer`, that should be loaded in as well.
#
from transformers import *

tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2')

/home/avik/miniconda3/envs/crows/lib/python3.8/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
loading file vocab.json from cache at /home/avik/.cache/huggingface/hub/models--distilgpt2/snapshots/38cc92ec43315abd5136313225e95acc5986876c/vocab.json
loading file merges.txt from cache at /home/avik/.cache/huggingface/hub/models--distilgpt2/snapshots/38cc92ec43315abd5136313225e95acc5986876c/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /home/avik/.cache/huggingface/hub/models--distilgpt2/snapshots/38cc92ec43315abd5136313225e95acc5986876c/config.json
Model config GPT2Config {
  "_name_or_path": "d

In [2]:
#
## Your code for initializing the rnn model(s)
#
# The Gulordava LSTM model can be found here: 
# https://drive.google.com/file/d/19Lp3AM4NEPycp_IBgoHfLc_V456pmUom/view?usp=sharing
# You can read more about this model in the original paper here: https://arxiv.org/pdf/1803.11138.pdf
#
# N.B: I have altered the RNNModel code to only output the hidden states that you are interested in.
# If you want to do more experiments with this model you could have a look at the original code here:
# https://github.com/facebookresearch/colorlessgreenRNNs/blob/master/src/language_models/model.py
#
from collections import defaultdict
from lstm.model import RNNModel
import torch


model_location = './state_dict.pt'  # <- point this to the location of the Gulordava .pt file
lstm = RNNModel('LSTM', 50001, 650, 650, 2)
lstm.load_state_dict(torch.load(model_location))


# This LSTM does not use a Tokenizer like the Transformers, but a Vocab dictionary that maps a token to an id.
with open('lstm/vocab.txt') as f:
    w2i = {w.strip(): i for i, w in enumerate(f)}

vocab = defaultdict(lambda: w2i["<unk>"])
vocab.update(w2i)

It is a good idea that before you move on, you try to feed some text to your LMs; and check if everything works accordingly. 

# Data

For this assignment you will train your probes on __treebank__ corpora. A treebank is a corpus that has been *parsed*, and stored in a representation that allows the parse tree to be recovered. Next to a parse tree, treebanks also often contain information about part-of-speech tags, which is exactly what we are after now.

The treebank you will use for now is part of the Universal Dependencies project. I provide a sample of this treebank as well, so you can test your setup on that before moving on to larger amounts of data.

Make sure you accustom yourself to the format that is created by the `conllu` library that parses the treebank files before moving on. For example, make sure you understand how you can access the pos tag of a token, or how to cope with the tree structure that is formed using the `to_tree()` functionality.

In [3]:
# READ DATA
from typing import List
from conllu import parse_incr, TokenList


# If stuff like `: str` and `-> ..` seems scary, fear not! 
# These are type hints that help you to understand what kind of argument and output is expected.
def parse_corpus(filename: str) -> List[TokenList]:
    data_file = open(filename, encoding="utf-8")

    ud_parses = list(parse_incr(data_file))
    
    return ud_parses


ud_parses = parse_corpus('data/sample/en_ewt-ud-train.conllu')


# Generating Representations

We now have our data all set, our models are running and we are good to go!

The next step is now to create the model representations for the sentences in our corpora. Once we have generated these representations we can store them, and train additional diagnostic (/probing) classifiers on top of the representations.

There are a few things you should keep in mind here. Read these carefully, as these tips will save you a lot of time in your implementation.
1. Transformer models make use of Byte-Pair Encodings (BPE), that chunk up a piece of next in subword pieces. For example, a word such as "largely" could be chunked up into "large" and "ly". We are interested in probing linguistic information on the __word__-level. Therefore, we will follow the suggestion of Hewitt et al. (2019a, footnote 4), and create the representation of a word by averaging over the representations of its subwords. So the representation of "largely" becomes the average of that of "large" and "ly".


2. Subword chunks never overlap multiple tokens. In other words, say we have a phrase like "None of the", then the tokenizer might chunk that into "No"+"ne"+" of"+" the", but __not__ into "No"+"ne o"+"f the", as those chunks overlap multiple tokens. This is great for our setup! Otherwise it would have been quite challenging to distribute the representation of a subword over the 2 tokens it belongs to.


3. **Important**: If you closely examine the provided treebank, you will notice that some tokens are split up into multiple pieces, that each have their own POS-tag. For example, in the first sentence the word "Al-Zaman" is split into "Al", "-", and "Zaman". In such cases, the conllu `TokenList` format will add the following attribute: `('misc', OrderedDict([('SpaceAfter', 'No')]))` to these tokens. Your model's tokenizer does not need to adhere to the same tokenization. E.g., "Al-Zaman" could be split into "Al-"+"Za"+"man", making it hard to match the representations with their correct pos-tag. Therefore I recommend you to not tokenize your entire sentence at once, but to do this based on the chunking of the treebank. <br /><br />
Make sure to still incoporate the spaces in a sentence though, as these are part of the BPE of the tokenizer. That is, the tokenizer uses a different token id for `"man"`, than it does for `" man"`: the former could be part of `" woman"`=`" wo`"+`"man"`, whereas the latter would be the used in case *man* occurs at the start of a word. The tokenizer for GPT-2 adds spaces at the start of a token (represented as a `Ġ` symbol). This means that you should keep track whether the previous token had the `SpaceAfter` attribute set to `'No'`: in case it did not, you should manually prepend a `" "` ahead of the token.


4. The LSTM LM does not have the issues related to subwords, but is far more restricted in its vocabulary. Make sure you keep the above points in mind though, when creating the LSTM representations. You might want to write separate functions for the LSTM, but that is up to you.


5. The huggingface transformer models don't return the hidden state by default. To achieve this you can pass `output_hidden_states=True` to a model forward pass. The hidden states are then returned for all intermediate layers as well, the latest entry in this list corresponds to the top layer.


6. **N.B.**: Make sure that when you run a sentence through your model, you do so within a `with torch.no_grad():` block, and that you have run `model.eval()` beforehand as well (to disable dropout).


7. **N.B.**: Make sure to use a token's ``["form"]`` attribute, and not the ``["lemma"]``, as the latter will stem any relevant morphological information from the token. We don't want this, because we want to feed well-formed, grammatical sentences to our model.


I would like to stress that if you feel hindered in any way by the simple code structure that is presented here, you are free to modify it :-) Just make sure it is clear to an outsider what you're doing, some helpful comments never hurt.

In [162]:
# FETCH SENTENCE REPRESENTATIONS
from torch import Tensor
import pickle


# Should return a tensor of shape (num_tokens_in_corpus, representation_size)
# Make sure you correctly average the subword representations that belong to 1 token!

def get_lstm_representations(ud_parses, model, tokenizer):
    model.eval()

    with torch.no_grad():
        hidden = model.init_hidden(1)
        arr = []
        for sent in ud_parses:
            for token in sent:
                form = token["form"]
                if form in tokenizer:
                    arr.append(tokenizer[form])
                else:
                    arr.append(tokenizer["<unk>"])
        ids = torch.tensor(arr)
        rep = model(ids.unsqueeze(0), hidden)
        rep = rep.squeeze(0)
        return rep
    

def get_gpt_representations(ud_parses, model, tokenizer):
    model.eval()
    
    with torch.no_grad():
        all_sentence_reps = []
        for sentence in ud_parses:
            reformed_sent = ""
            no_of_original_tokens = len(sentence)
            
            for tok_idx in range(no_of_original_tokens):
                token = sentence[tok_idx]['form']
                
                if (
                    (
                    sentence[tok_idx]['misc'] is None or 
                    sentence[tok_idx]['form'] in ['"', "[", "]", "(", ")", ".", ",", "?", "'", "other", "can"]
                    ) and 
                    tok_idx != no_of_original_tokens-1
                ):
                    reformed_sent = reformed_sent + token + " "
                else:
                    reformed_sent = reformed_sent + token
            
            inputs = tokenizer(reformed_sent, return_tensors="pt")
                
            outputs = model(**inputs, output_hidden_states=True)
            final_reps = outputs.hidden_states[-1][0]

            idx2 = 0
            start_again = True
            combined_reps = []
            old_token = ""

            for idx, input_id in enumerate(inputs['input_ids'][0]):
                token = tokenizer.convert_ids_to_tokens(input_id.item())

                #### Handling n't ####
                
                if idx2 != no_of_original_tokens-1 and sentence[idx2+1]['form'] == "n't":
                    sentence[idx2]['form'] += sentence[idx2+1]['form'][0]
                    sentence[idx2+1]['form'] = sentence[idx2+1]['form'][1:]

                ######################
                
                if start_again:
                    combined_word_piece = ""
                    curr_rep = []
                    start_again = False

                #### Handling Déjà ####

                if token == 'Ã©':
                    token = 'é'
                if token == 'Ãł':
                    token = 'à'

                #######################
                
                if token[0] == 'Ġ':
                    cur_word_piece = token[1:]
                else:
                    cur_word_piece = token

                #### Handling 's ####
                
                if token == 'âĢ':
                    for _idx in range(3):
                        curr_rep.append(final_reps[idx+_idx])
                    curr_rep = torch.stack(curr_rep)
                    combined_reps.append(curr_rep.mean(0))
                    start_again = True
                    old_token = token
                    idx2 += 1
                    continue
                elif old_token == 'âĢ' and token == 'Ļ':
                    start_again = True
                    old_token = token
                    continue
                elif old_token == 'Ļ' and token == 's':
                    start_again = True
                    old_token = token
                    continue

                #####################

                combined_word_piece = combined_word_piece + cur_word_piece
                curr_rep.append(final_reps[idx])

                if combined_word_piece == sentence[idx2]['form']:
                    curr_rep = torch.stack(curr_rep)
                    combined_reps.append(curr_rep.mean(0))
                    start_again = True
                    old_token = token
                    idx2 += 1
        
            sentence_rep = torch.stack(combined_reps)

            #### DEBUG CODE ####

            if sentence_rep.size(0) != no_of_original_tokens:
                print(f"SENT REP SIZE: {sentence_rep.size(0)} ORIG SIZE: {no_of_original_tokens}")
                print("SENT REC: ", reformed_sent)

                for tok_idx in range(no_of_original_tokens):
                    token = sentence[tok_idx]['form']
                    print(token, end=' ')
                    
                print()
                inputs = tokenizer(reformed_sent, return_tensors="pt")

                for idx, input_id in enumerate(inputs['input_ids'][0]):
                    token = tokenizer.convert_ids_to_tokens(input_id.item())
                    print(token, end=' ')
                print()

            ####################
            
            all_sentence_reps.append(sentence_rep)
        return torch.cat(all_sentence_reps)


def fetch_sen_reps(ud_parses: List[TokenList], model, tokenizer) -> Tensor:    
    if isinstance(model, GPT2LMHeadModel):
        rep = get_gpt_representations(ud_parses, model, tokenizer)
    elif isinstance(model, RNNModel):
        rep = get_lstm_representations(ud_parses, model, tokenizer)
    else:
        print("NOT A SUPPORTED MODEL!!")
        return None
    
    return rep

To validate your activation extraction procedure I have set up the following assertion function as a sanity check. It compares your representation of the first sentence in the corpus against a pickled version of mine. 

For this I used `distilgpt2`.

In [163]:
def error_msg(model_name, gold_embs, embs, i2w):
    with open(f'{model_name}_tokens1.pickle', 'rb') as f:
        sen_tokens = pickle.load(f)
        
    diff = torch.abs(embs - gold_embs)
    max_diff = torch.max(diff)
    avg_diff = torch.mean(diff)
    
    print(f"{model_name} embeddings don't match!")
    print(f"Max diff.: {max_diff:.4f}\nMean diff. {avg_diff:.4f}")

    print("\nCheck if your tokenization matches with the original tokenization:")
    for idx in sen_tokens.squeeze():
        if isinstance(i2w, list):
            token = i2w[idx]
        else:
            token = i2w.convert_ids_to_tokens(idx.item())
        print(f"{idx:<6} {token}")


def assert_sen_reps(model, tokenizer, lstm, vocab):
    with open('distilgpt2_emb1.pickle', 'rb') as f:
        distilgpt2_emb1 = pickle.load(f)
        
    with open('lstm_emb1.pickle', 'rb') as f:
        lstm_emb1 = pickle.load(f)
       
    
    corpus = parse_corpus('data/sample/en_ewt-ud-train.conllu')[:1]
    
    own_distilgpt2_emb1 = fetch_sen_reps(corpus, model, tokenizer)
    own_lstm_emb1 = fetch_sen_reps(corpus, lstm, vocab)
    
    assert distilgpt2_emb1.shape == own_distilgpt2_emb1.shape, \
        f"Distilgpt2 shape mismatch: {distilgpt2_emb1.shape} (gold) vs. {own_distilgpt2_emb1.shape} (yours)"
    assert lstm_emb1.shape == own_lstm_emb1.shape, \
        f"LSTM shape mismatch: {lstm_emb1.shape} (gold) vs. {own_lstm_emb1.shape} (yours)"

    if not torch.allclose(distilgpt2_emb1, own_distilgpt2_emb1, rtol=1e-3, atol=1e-3):
        error_msg("distilgpt2", distilgpt2_emb1, own_distilgpt2_emb1, tokenizer)
    if not torch.allclose(lstm_emb1, own_lstm_emb1, rtol=1e-3, atol=1e-3):
        error_msg("lstm", lstm_emb1, own_lstm_emb1, list(vocab.keys()))


assert_sen_reps(model, tokenizer, lstm, vocab)

Next, we should define a function that extracts the corresponding POS labels for each activation, which we do based on the **``"upostag"``** attribute of a token (so not the ``xpostag`` attribute). These labels will be transformed to a tensor containing the label index for each item.

In [164]:
# FETCH POS LABELS

from typing import Optional, Dict, Tuple
# Should return a tensor of shape (num_tokens_in_corpus,)
# Make sure that when fetching these pos tags for your train/dev/test corpora you share the label vocabulary.
def fetch_pos_tags(ud_parses: List[TokenList], pos_vocab: Optional[Dict[str, int]] = None) -> Tuple[Tensor, Dict[str, int]]:
    pos_tags = []
    for sent in ud_parses:
        for token in sent:
            pos = token["upostag"]
            if pos_vocab is None:
                pos_vocab = {}
            if pos not in pos_vocab:
                new_id =  len(pos_vocab)
                pos_vocab[pos] = new_id
                pos_tags.append(new_id)
            elif pos in pos_vocab:
                old_id = pos_vocab[pos]
                pos_tags.append(old_id)
            else: 
                pass
    return torch.tensor(pos_tags), pos_vocab
                
                

Finally, we can combine all these methods to retrieve the representations (`fetch_sen_reps`) and the corresponding labels (`fetch_pos_tags`). If you are still debugging and testing your setup you can set the `use_sample` variable to `True`, and once everything works you can extract the full corpus by setting it to `False`.

The reason we pass the `train_vocab` to the data creation of the `dev` and `test` data is that we want to use the same label vocabulary across the different train/dev/test splits.

In [165]:
import os

# Function that combines the previous functions, and creates 2 tensors for a .conllu file: 
# 1 containing the token representations, and 1 containing the (tokenized) pos_tags.

def create_data(ud_parses, filename: str, lm, w2i, pos_vocab=None):
    ud_parses = parse_corpus(filename)
    
    sen_reps = fetch_sen_reps(ud_parses, lm, w2i)
    pos_tags, pos_vocab = fetch_pos_tags(ud_parses, pos_vocab=pos_vocab)
    
    return sen_reps, pos_tags, pos_vocab


lm = model  # or `lstm`
w2i = tokenizer  # or `vocab`
# lm = lstm
# w2i = vocab
use_sample = True

train_x, train_y, train_vocab = create_data(
    ud_parses,
    os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-train.conllu'),
    lm, 
    w2i
)

dev_x, dev_y, _ = create_data(
    ud_parses,
    os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-dev.conllu'),
    lm, 
    w2i,
    pos_vocab=train_vocab
)

test_x, test_y, _ = create_data(
    ud_parses,
    os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-test.conllu'),
    lm,
    w2i,
    pos_vocab=train_vocab
)

# Diagnostic Classification

We now have our models, our data, _and_ our representations all set! Hurray, well done. We can finally move onto the cool stuff, i.e. training the diagnostic classifiers (DCs).

DCs are simple in their complexity on purpose. To read more about why this is the case you could already have a look at the "Designing and Interpreting Probes with Control Tasks" by Hewitt and Liang (esp. Sec. 3.2).

A simple linear classifier will suffice for now, don't bother with adding fancy non-linearities to it.

I am personally a fan of the `skorch` library, that provides `sklearn`-like functionalities for training `torch` models, but you are free to train your dc using whatever method you prefer.

As this is an Artificial Intelligence master and you have all done ML1 + DL, I expect you to use your train/dev/test splits correctly ;-)

In [166]:
import torch
from torch import nn
import torch.nn.functional as F

# ! pip install skorch
from skorch import NeuralNetClassifier

In [167]:
# DIAGNOSTIC CLASSIFIER
class ClassifierModule(nn.Module):
    def __init__(
            self,
            num_units=15,
    ):
        super(ClassifierModule, self).__init__()
        self.num_units = num_units
        self.output = nn.Linear(768, num_units)


    def forward(self, X, **kwargs):
        X = self.output(X)
        X = F.softmax(X, dim=-1)
        return X

net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=2500,
    lr=0.001)

In [168]:
net.fit(train_x, train_y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        4.7775       0.1605        4.0082  0.0366
      2        4.3028       0.1605        4.3227  0.0417
      3        3.4548       0.1931        3.9069  0.0337
      4        3.6928       0.1974        3.2300  0.0443
      5        3.4566       0.2061        3.5351  0.0435
      6        3.3304       0.1714        4.1787  0.0358
      7        3.0266       0.1584        4.0490  0.0376
      8        3.1851       0.1692        3.3491  0.0330
      9        2.7630       0.1518        3.1877  0.0321
     10        2.8911       0.2386        3.1617  0.0325
     11        2.9318       0.2017        3.0557  0.0281
     12        2.8477       0.2603        2.6645  0.0296
     13        2.7059       0.2668        2.6235  0.0342
     14        2.5780       0.2321        2.8772  0.0354
     15        2.6100       0.2256        2.5514  0.0375
     16        2.5962       0.2

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (output): Linear(in_features=768, out_features=15, bias=True)
  ),
)

In [169]:
net.score(dev_x,dev_y)

0.8524871355060034

In [170]:
net.score(test_x, test_y)

0.8282740676496098

# Trees

For our gold labels, we need to recover the node distances from our parse tree. For this we will use the functionality provided by `ete3`, that allows us to compute that directly. I have provided code that transforms a `TokenTree` to a `Tree` in `ete3` format.

In [171]:
# In case you want to transform your conllu tree to an nltk.Tree, for better visualisation

def rec_tokentree_to_nltk(tokentree):
    token = tokentree.token["form"]
    tree_str = f"({token} {' '.join(rec_tokentree_to_nltk(t) for t in tokentree.children)})"

    return tree_str


def tokentree_to_nltk(tokentree):
    from nltk import Tree as NLTKTree

    tree_str = rec_tokentree_to_nltk(tokentree)

    return NLTKTree.fromstring(tree_str)

In [172]:
# !pip install ete3
from ete3 import Tree as EteTree


class FancyTree(EteTree):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, format=1, **kwargs)
        
    def __str__(self):
        return self.get_ascii(show_internal=True)
    
    def __repr__(self):
        return str(self)


def rec_tokentree_to_ete(tokentree):
    idx = str(tokentree.token["id"])
    children = tokentree.children
    if children:
        return f"({','.join(rec_tokentree_to_ete(t) for t in children)}){idx}"
    else:
        return idx
    
def tokentree_to_ete(tokentree):
    newick_str = rec_tokentree_to_ete(tokentree)

    return FancyTree(f"{newick_str};")

In [173]:
# Let's check if it works!
# We can read in a corpus using the code that was already provided, and convert it to an ete3 Tree.

def parse_corpus(filename):
    from conllu import parse_incr

    data_file = open(filename, encoding="utf-8")

    ud_parses = list(parse_incr(data_file))
    
    return ud_parses

corpus = parse_corpus('data/sample/en_ewt-ud-train.conllu')
item = corpus[0]
tokentree = item.to_tree()
ete3_tree = tokentree_to_ete(tokentree)
print(ete3_tree)


   /-2
  |
  |--3
  |
  |--4
  |
  |   /6 /-5
  |  |
  |  |   /-9
  |  |  |
  |  |  |--10
  |  |  |
  |  |  |--11
  |  |-8|
  |  |  |--12
  |-7|  |
  |  |  |--13
  |  |  |
  |  |   \15/-14
-1|  |
  |  |   /-16
  |  |  |
  |  |  |--17
  |  |  |
  |   \18   /-19
  |     |  |
  |     |  |--20
  |     |  |
  |     |  |-23/-22
  |      \21
  |        |--24
  |        |
  |        |   /-25
  |        |  |
  |         \28--26
  |           |
  |            \-27
  |
   \-29


As you can see we label a token by its token id (converted to a string). Based on these id's we are going to retrieve the node distances.

To create the true distances of a parse tree in our treebank, we are going to use the `.get_distance` method that is provided by `ete3`: http://etetoolkit.org/docs/latest/tutorial/tutorial_trees.html#working-with-branch-distances

We will store all these distances in a `torch.Tensor`.

Please fill in the gap in the following method. I recommend you to have a good look at Hewitt's blog post  about these node distances.

In [174]:
def create_gold_distances(corpus):
    all_distances = []

    for item in (corpus):
        tokentree = item.to_tree()
        ete_tree = tokentree_to_ete(tokentree)
 
        sen_len = len(ete_tree.search_nodes())
        distances = torch.zeros((sen_len, sen_len))
        for src_node in range(1, sen_len+1):
            for target_node in range(1, sen_len+1):
                if distances[src_node-1][target_node-1] == 0:
                     s = ete_tree&str(src_node)
                     t  = ete_tree&str(target_node)
                     dist = s.get_distance(t)
                     distances[src_node-1][target_node-1] = dist
                     distances[target_node-1][src_node-1] = dist

        # Your code for computing all the distances comes here.

        all_distances.append(distances)

    return all_distances

The next step is now to do the previous step the other way around. After all, we are mainly interested in predicting the node distances of a sentence, in order to recreate the corresponding parse tree.

Hewitt et al. reconstruct a parse tree based on a _minimum spanning tree_ (MST, https://en.wikipedia.org/wiki/Minimum_spanning_tree). Fortunately for us, we can simply import a method from `scipy` that retrieves this MST.

In [175]:
from scipy.sparse.csgraph import minimum_spanning_tree
import torch


def create_mst(distances):
    distances = torch.triu(distances).detach().numpy()
    mst = minimum_spanning_tree(distances).toarray()
    mst[mst>0] = 1.
    return mst

Let's have a look at what this looks like, by looking at a relatively short sentence in the sample corpus.

If your addition to the `create_gold_distances` method has been correct, you should be able to run the following snippet. This then shows you the original parse tree, the distances between the nodes, and the MST that is retrieved from these distances. Can you spot the edges in the MST matrix that correspond to the edges in the parse tree?

In [176]:
item = corpus[5]
tokentree = item.to_tree()
ete3_tree = tokentree_to_ete(tokentree)
print(ete3_tree, '\n')

gold_distance = create_gold_distances(corpus[5:6])[0]

mst = create_mst(gold_distance)


   /2 /-1
  |
  |--3
  |
  |--4
  |
  |   /-6
  |  |
-5|  |--7
  |-8|
  |  |   /-9
  |  |  |
  |   \12--10
  |     |
  |      \-11
  |
   \-13 

tensor([[0., 1., 3., 3., 2., 4., 4., 3., 5., 5., 5., 4., 3.],
        [1., 0., 2., 2., 1., 3., 3., 2., 4., 4., 4., 3., 2.],
        [3., 2., 0., 2., 1., 3., 3., 2., 4., 4., 4., 3., 2.],
        [3., 2., 2., 0., 1., 3., 3., 2., 4., 4., 4., 3., 2.],
        [2., 1., 1., 1., 0., 2., 2., 1., 3., 3., 3., 2., 1.],
        [4., 3., 3., 3., 2., 0., 2., 1., 3., 3., 3., 2., 3.],
        [4., 3., 3., 3., 2., 2., 0., 1., 3., 3., 3., 2., 3.],
        [3., 2., 2., 2., 1., 1., 1., 0., 2., 2., 2., 1., 2.],
        [5., 4., 4., 4., 3., 3., 3., 2., 0., 2., 2., 1., 4.],
        [5., 4., 4., 4., 3., 3., 3., 2., 2., 0., 2., 1., 4.],
        [5., 4., 4., 4., 3., 3., 3., 2., 2., 2., 0., 1., 4.],
        [4., 3., 3., 3., 2., 2., 2., 1., 1., 1., 1., 0., 3.],
        [3., 2., 2., 2., 1., 3., 3., 2., 4., 4., 4., 3., 0.]]) 

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0

Now that we are able to map edge distances back to parse trees, we can create code for our quantitative evaluation. For this we will use the Undirected Unlabeled Attachment Score (UUAS), which is expressed as:

$$\frac{\text{number of predicted edges that are an edge in the gold parse tree}}{\text{number of edges in the gold parse tree}}$$

To do this, we will need to obtain all the edges from our MST matrix. Note that, since we are using undirected trees, that an edge can be expressed in 2 ways: an edge between node $i$ and node $j$ is denoted by both `mst[i,j] = 1`, or `mst[j,i] = 1`.

You will write code that computes the UUAS score for a matrix of predicted distances, and the corresponding gold distances. I recommend you to split this up into 2 methods: 1 that retrieves the edges that are present in an MST matrix, and one general method that computes the UUAS score.

In [177]:
def edges(mst):
    edges = set()

    edges_list = []
    for i, row in enumerate(mst):
        for j, val in enumerate(mst[i]):
            if int(val) == 1 and i<j: # i < j ensures that (1,2) is added and not (2,1) undirectional case
                edges_list.append((i,j))
                
    edges = set(edges_list)
    return edges


def calc_uuas(pred_distances, gold_distances):
    num, denom = 0, 0
    uuas = 0
    
    for pred_matrix, gold_matrix in zip(pred_distances, gold_distances):
        pred_mst = create_mst(pred_matrix)
        gold_mst = create_mst(gold_matrix)
        pred_edges = edges(pred_mst)
        gold_edges = edges(gold_mst)
        edges_pg = pred_edges & gold_edges
        num += len(edges_pg)
        denom += len(gold_edges)

    uuas = num/denom
    return uuas


# Structural Probes

We now have everything in place to start doing the actual exciting stuff: training our structural probe!
    
To make life easier for you, we will simply take the `torch` code for this probe from John Hewitt's repository. This allows you to focus on the training regime from now on.

In [193]:
import torch.nn as nn
import torch


class StructuralProbe(nn.Module):
    """ Computes squared L2 distance after projection by a matrix.
    For a batch of sentences, computes all n^2 pairs of distances
    for each sentence in the batch.
    """
    def __init__(self, model_dim, rank, device="cpu"):
        super().__init__()
        self.probe_rank = rank
        self.model_dim = model_dim
        
        self.proj = nn.Parameter(data = torch.zeros(self.model_dim, self.probe_rank))
        
        nn.init.uniform_(self.proj, -0.05, 0.05)
        self.to(device)

    def forward(self, batch):
        """ Computes all n^2 pairs of distances after projection
        for each sentence in a batch.
        Note that due to padding, some distances will be non-zero for pads.
        Computes (B(h_i-h_j))^T(B(h_i-h_j)) for all i,j
        Args:
          batch: a batch of word representations of the shape
            (batch_size, max_seq_len, representation_dim)
        Returns:
          A tensor of distances of shape (batch_size, max_seq_len, max_seq_len)
        """
        transformed = torch.matmul(batch, self.proj)
        
        batchlen, seqlen, rank = transformed.size()
        
        transformed = transformed.unsqueeze(2)
        transformed = transformed.expand(-1, -1, seqlen, -1)
        transposed = transformed.transpose(1,2)
        
        diffs = transformed - transposed
        
        squared_diffs = diffs.pow(2)
        squared_distances = torch.sum(squared_diffs, -1)

        return squared_distances

    
class L1DistanceLoss(nn.Module):
    """Custom L1 loss for distance matrices."""
    def __init__(self):
        super().__init__()

    def forward(self, predictions, label_batch, length_batch):
        """ Computes L1 loss on distance matrices.
        Ignores all entries where label_batch=-1
        Normalizes first within sentences (by dividing by the square of the sentence length)
        and then across the batch.
        Args:
          predictions: A pytorch batch of predicted distances
          label_batch: A pytorch batch of true distances
          length_batch: A pytorch batch of sentence lengths
        Returns:
          A tuple of:
            batch_loss: average loss in the batch
            total_sents: number of sentences in the batch
        """
        labels_1s = (label_batch != -1).float()
        predictions_masked = predictions * labels_1s
        labels_masked = label_batch * labels_1s
        total_sents = torch.sum((length_batch != 0)).float()
        squared_lengths = length_batch.pow(2).float()

        if total_sents > 0:
            loss_per_sent = torch.sum(torch.abs(predictions_masked - labels_masked), dim=(1,2))
            normalized_loss_per_sent = loss_per_sent / squared_lengths
            batch_loss = torch.sum(normalized_loss_per_sent) / total_sents
        
        else:
            batch_loss = torch.tensor(0.0)
        
        return batch_loss, total_sents


I have provided a rough outline for the training regime that you can use. Note that the hyper parameters that I provide here only serve as an indication, but should be (briefly) explored by yourself.

As can be seen in Hewitt's code above, there exists functionality in the probe to deal with batched input. It is up to you to use that: a (less efficient) method can still incorporate batches by doing multiple forward passes for a batch and computing the backward pass only once for the summed losses of all these forward passes. (_I know, this is not the way to go, but in the interest of time that is allowed ;-), the purpose of the assignment is writing a good paper after all_).

In [217]:
from torch import optim
from tqdm import tqdm

'''
Similar to the `create_data` method of the previous notebook, I recommend you to use a method 
that initialises all the data of a corpus. Note that for your embeddings you can use the 
`fetch_sen_reps` method again. However, for the POS probe you concatenated all these representations into 
1 big tensor of shape (num_tokens_in_corpus, model_dim). 

The StructuralProbe expects its input to contain all the representations of 1 sentence, so I recommend you
to update your `fetch_sen_reps` method in a way that it is easy to retrieve all the representations that 
correspond to a single sentence.
''' 


def fetch_sen_reps(ud_parses: List[TokenList], model, tokenizer, concat) -> Tensor:    
    rep = []
    if isinstance(model, GPT2LMHeadModel):
        for ud_parse in ud_parses:
            rep.append(get_gpt_representations([ud_parse], model, tokenizer))
        if concat:
            rep = nn.utils.rnn.pad_sequence(rep, batch_first=True)
    elif isinstance(model, RNNModel):
        for ud_parse in ud_parses:
            rep.append(get_lstm_representations([ud_parse], model, tokenizer))
        if concat:
            rep = nn.utils.rnn.pad_sequence(rep, batch_first=True)
    else:
        print("NOT A SUPPORTED MODEL!!")
        return None
    
    return rep
    

def init_corpus(path, model, tokenizer, concat=False, cutoff=None):
    """ Initialises the data of a corpus.
    
    Parameters
    ----------
    path : str
        Path to corpus location
    concat : bool, optional
        Optional toggle to concatenate all the tensors
        returned by `fetch_sen_reps`.
    cutoff : int, optional
        Optional integer to "cutoff" the data in the corpus.
        This allows only a subset to be used, alleviating 
        memory usage.
    """
    corpus = parse_corpus(path)[:cutoff]

    embs = fetch_sen_reps(corpus, model, tokenizer, concat=concat)    
    gold_distances = create_gold_distances(corpus)

    lengths = [sent.size(0) for sent in gold_distances]
    maxlen = int(max(lengths))
    label_maxshape = [maxlen for _ in gold_distances[0].shape]
    labels = [-torch.ones(*label_maxshape) for _ in range(len(lengths))]

    for idx, gold_dist in enumerate(gold_distances):
        length = lengths[idx]
        labels[idx][:length,:length] = gold_dist
    
    labels = torch.stack(labels)

    return labels, embs, torch.Tensor(lengths)


# I recommend you to write a method that can evaluate the UUAS & loss score for the dev (& test) corpus.
# Feel free to alter the signature of this method.
def evaluate_probe(probe, loss_function, _data):
    # YOUR CODE HERE

    probe.eval()
    y, x, sent_lens = _data
    preds = probe(x)
    loss_score, _ = loss_function(preds, y, sent_lens)
    
    preds_new, y_new = [], []
    for i, length in enumerate(sent_lens):
        length = int(length)
        preds_resized, y_resized = preds[i, :length, :length], y[i, :length, :length]
        preds_new.append(preds_resized)
        y_new.append(y_resized)
 
    uuas_score = calc_uuas(preds_new, y_new)
    return loss_score, uuas_score


# Feel free to alter the signature of this method.
def train(_data, _dev_data, _test_data, epochs):
    emb_dim = 768
    rank = 64
    lr = 1e-1
    batch_size = 15

    probe = StructuralProbe(emb_dim, rank)
    optimizer = optim.Adam(probe.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5,patience=1)
    loss_function =  L1DistanceLoss()

    train_y, train_x, train_sent_lens = _data

    for epoch in range(epochs):

        for i in range(0, len(corpus), batch_size):
            probe.train()
            optimizer.zero_grad()

            # YOUR CODE FOR DOING A PROBE FORWARD PASS

            _train_batch = train_x[i:i+batch_size]
            _train_labels = train_y[i:i+batch_size]
            _train_lengths = train_sent_lens[i:i+batch_size]
            _preds = probe(_train_batch)
            batch_loss, total_sents = loss_function(_preds, _train_labels, _train_lengths)
            batch_loss.backward()
            optimizer.step()

        dev_loss, dev_uuas = evaluate_probe(probe, loss_function, _dev_data)
        print("Dev Loss: {}, Dev uuas: {}".format(dev_loss, dev_uuas))
        # Using a scheduler is up to you, and might require some hyper param fine-tuning
        scheduler.step(dev_loss)

    test_loss, test_uuas = evaluate_probe(probe,loss_function,_test_data)
    print("Test Loss: {}, Test uuas: {}".format(test_loss, test_uuas))
    return probe, loss_function

In [218]:
train_data_path = 'data/sample/en_ewt-ud-train.conllu'
dev_data_path = 'data/sample/en_ewt-ud-dev.conllu'
test_data_path = 'data/sample/en_ewt-ud-test.conllu'

lm = model  # `model` or `lstm`
w2i = tokenizer  # `tokenizer` or `vocab`

train_data = init_corpus(train_data_path, lm, w2i, concat=True)
dev_data = init_corpus(dev_data_path, lm, w2i, concat=True)
test_data = init_corpus(test_data_path, lm, w2i, concat=True)

In [219]:
probe, loss_fn = train(train_data, dev_data, test_data, 200)

Dev Loss: 1527.6253662109375, Dev uuas: 0.13261648745519714
Dev Loss: 967.2228393554688, Dev uuas: 0.1335125448028674
Dev Loss: 921.4500732421875, Dev uuas: 0.11827956989247312
Dev Loss: 560.2213134765625, Dev uuas: 0.12096774193548387
Dev Loss: 348.4796447753906, Dev uuas: 0.11827956989247312
Dev Loss: 276.2138366699219, Dev uuas: 0.13082437275985664
Dev Loss: 190.75758361816406, Dev uuas: 0.10483870967741936
Dev Loss: 132.82565307617188, Dev uuas: 0.1460573476702509
Dev Loss: 105.5123062133789, Dev uuas: 0.12813620071684587
Dev Loss: 81.04525756835938, Dev uuas: 0.13978494623655913
Dev Loss: 83.60990142822266, Dev uuas: 0.11290322580645161
Dev Loss: 96.96717071533203, Dev uuas: 0.12813620071684587
Dev Loss: 59.1540412902832, Dev uuas: 0.10215053763440861
Dev Loss: 33.56718444824219, Dev uuas: 0.11559139784946236
Dev Loss: 18.564495086669922, Dev uuas: 0.12544802867383512
Dev Loss: 10.33349323272705, Dev uuas: 0.13799283154121864
Dev Loss: 6.225638389587402, Dev uuas: 0.12724014336917

## LaTeX trees

For your report you might want to add some of those fancy dependency tree plots like those of Figure 2 in the Structural Probing paper. For that you can use the following code, that outputs the corresponding LaTeX markup.

**N.B.**: for the latex tikz tree the first token in a sentence has index 1 (instead of 0), so take that into account with the predicted and gold edges that you pass to the method.

In [25]:
def print_tikz(predicted_edges, gold_edges, words):
    """ Turns edge sets on word (nodes) into tikz dependency LaTeX.
    Parameters
    ----------
    predicted_edges : Set[Tuple[int, int]]
        Set (or list) of edge tuples, as predicted by your probe.
    gold_edges : Set[Tuple[int, int]]
        Set (or list) of gold edge tuples, as obtained from the treebank.
    words : List[str]
        List of strings representing the tokens in the sentence.
    """

    string = """\\begin{dependency}[hide label, edge unit distance=.5ex]
    \\begin{deptext}[column sep=0.05cm]
    """

    string += (
        "\\& ".join([x.replace("$", "\$").replace("&", "+") for x in words])
        + " \\\\\n"
    )
    string += "\\end{deptext}" + "\n"
    for i_index, j_index in gold_edges:
        string += "\\depedge[-]{{{}}}{{{}}}{{{}}}\n".format(i_index, j_index, ".")
    for i_index, j_index in predicted_edges:
        string += f"\\depedge[-,edge style={{red!60!}}, edge below]{{{i_index}}}{{{j_index}}}{{.}}\n"
    string += "\\end{dependency}\n"
    print(string)

In [29]:
probe.eval()
y, x, sent_lens = test_data
preds = probe(x)

corpus = parse_corpus(test_data_path)[0]

preds_new, y_new = [], []
for i, length in enumerate(sent_lens):
    length = int(length)
    preds_resized, y_resized = preds[i, :length, :length], y[i, :length, :length]
    preds_new.append(preds_resized)
    y_new.append(y_resized)
    break

for pred_matrix, gold_matrix in zip(preds_new, y_new):
    pred_mst = create_mst(pred_matrix)
    gold_mst = create_mst(gold_matrix)
    pred_edges = edges(pred_mst)
    gold_edges = edges(gold_mst)

words = []
for sent in ud_parses:
    for token in sent:
        words.append(token["form"])

print_tikz(pred_edges, gold_edges, words)

\begin{dependency}[hide label, edge unit distance=.5ex]
    \begin{deptext}[column sep=0.05cm]
    Al\& -\& Zaman\& :\& American\& forces\& killed\& Shaikh\& Abdullah\& al\& -\& Ani\& ,\& the\& preacher\& at\& the\& mosque\& in\& the\& town\& of\& Qaim\& ,\& near\& the\& Syrian\& border\& .\& [\& This\& killing\& of\& a\& respected\& cleric\& will\& be\& causing\& us\& trouble\& for\& years\& to\& come\& .\& ]\& DPA\& :\& Iraqi\& authorities\& announced\& that\& they\& had\& busted\& up\& 3\& terrorist\& cells\& operating\& in\& Baghdad\& .\& Two\& of\& them\& were\& being\& run\& by\& 2\& officials\& of\& the\& Ministry\& of\& the\& Interior\& !\& The\& MoI\& in\& Iraq\& is\& equivalent\& to\& the\& US\& FBI\& ,\& so\& this\& would\& be\& like\& having\& J.\& Edgar\& Hoover\& unwittingly\& employ\& at\& a\& high\& level\& members\& of\& the\& Weathermen\& bombers\& back\& in\& the\& 1960s\& .\& The\& third\& was\& being\& run\& by\& the\& head\& of\& an\& investment\& firm\& .\& You\&